### Cumulative Pipeline Creation
#### Objective: 
From the least cost pipeline network created in `LC-Pipeline-Construction`, create nodes where pipelines from different sources merge and label each pipeline segment with the cumulative amound of methane transported through it. 

#### Workflow:
1. Create a flow direction raster from the LCP and the connection point feature classes:
    * Convert the LCP feature class to a raster, using the "Order" attribute to assign values
    * Compute cost distance and cost traceback rasters away from the connection point. 
    * Convert the traceback raster computed above into a flow direction raster.
2. Create a flow accumulation weight raster from the biogas sources:
    * Convert the traceback raster to a polyline feature class (via stream to feature)
    * Snap the Biogas Source features to the polyline feature class
    * Convert the snapped biogas source to a raster, using the Estimated throughput as the value
    * Set null values in the above raster that fall along the flow direction raster to zero. 
3. Compute weighted flow accumulation and convert back to a polyline feature class
    * Compute weighted flow accumulation using the flow direction and biogas source weights.
    * Convert the weighted flow accumulation to a polyline feature class via stream to feature.
    
#### Data inputs:
* LCP polyline feature class
* Connection point feature class

In [8]:
#Import packages
import arcpy
from arcpy import sa
arcpy.CheckOutExtension('Spatial')

'CheckedOut'

In [2]:
#Set folders
import os
scriptsDir = os.getcwd()
rootDir = os.path.dirname(scriptsDir)
scratchDir = os.path.join(rootDir,"scratch")
scratchGDB = os.path.join(scratchDir,'scratch.gdb')

In [53]:
#Set inputs
lcp_fc = os.path.join(scratchDir,'LCPNetwork.shp')
connect_fc =  os.path.join(scratchDir,'ConnectionPoint.shp')
biogas_fc =  os.path.join(rootDir,'OptimizeViewer','OptimizeViewer.gdb','Biogas_Prj')
#Set the output feature class name
lcpnet_fc = os.path.join(scratchDir,'LCPNetwork_Flow.shp')

In [35]:
#Set arcpy environments
arcpy.env.overwriteOutput = True
arcpy.env.workspace =  scratchGDB
arcpy.env.scratchWorkspace = scratchGDB
arcpy.env.cellSize = 500
arcpy.env.extent = lcp_fc

In [48]:
#Set temp filenames
lcp_raster = 'LCP_raster'
cdist_raster = 'LCP_cdist'
blink_raster = 'LCP_blink'
lcp_lines = 'LCP_pathline'
biogas_snapped = 'Biogas_Snapped'
bgSrc_raster = 'Biogas_Raster'

In [6]:
#Convert LCP fc to a raster at 100 m cell size
lcp_raster = arcpy.conversion.PolylineToRaster(in_features=lcp_fc,
                                               value_field='Order',
                                               out_rasterdataset=lcp_raster,
                                               priority_field='Order',
                                               cellsize='500')

In [11]:
#Compute cost distance to connection point
cdist_output = arcpy.sa.CostDistance(in_source_data=connect_fc,
                                     in_cost_raster=lcp_raster,
                                     out_backlink_raster = blink_raster)

In [13]:
#Convert backlink to flow direction
flow_dir = sa.Int(sa.Exp2(blink_raster) / 2)

In [22]:
#Create "stream polyline" from flow dir raster
stream_fc = sa.StreamToFeature(in_stream_raster=flow_dir,
                               in_flow_direction_raster=flow_dir,
                               out_polyline_features=lcp_lines,
                               simplify='NO_SIMPLIFY').getoutput(0)

In [57]:
#Snap biogas source points to the network
#Copy the features first...
arcpy.management.CopyFeatures(in_features=biogas_fc, out_feature_class=biogas_snapped)
#Snap the features...
arcpy.edit.Snap(biogas_snapped,[[stream_fc,'EDGE','1500 Meters']])

<Result 'C:\\Workspace\\Gits\\BioGas_optimization\\scratch\\scratch.gdb\\Biogas_Snapped'>

In [58]:
#Convert snapped back to raster, using the "Total_Potential_Methane_Yield_1" as the value
arcpy.conversion.PointToRaster(in_features=biogas_snapped,
                               value_field='Total_Potential_Methane_Yield_1',
                               out_rasterdataset=bgSrc_raster,
                               cell_assignment='SUM')

<Result 'C:\\Workspace\\Gits\\BioGas_optimization\\scratch\\scratch.gdb\\Biogas_Raster'>

In [59]:
#Set null values to zero
bgSrc_weight_raster = sa.Con(sa.IsNull(bgSrc_raster),0,bgSrc_raster)

In [60]:
#Compute flow accumulation 
bgSrc_accum = sa.FlowAccumulation(in_flow_direction_raster=flow_dir,
                                  in_weight_raster=bgSrc_weight_raster,
                                  data_type='float',
                                  flow_direction_type='D8')

In [61]:
#Add the original weights and convert to integer
bcSrc_accum_2 = sa.Int(bgSrc_accum + bgSrc_weight_raster)

In [63]:
#Convert accumulation to a feature class
final_fc = sa.StreamToFeature(in_stream_raster=bcSrc_accum_2,
                               in_flow_direction_raster=flow_dir,
                               out_polyline_features=lcpnet_fc,
                               simplify='NO_SIMPLIFY').getoutput(0)

In [64]:
bcSrc_accum_2.save('LCPFlow_raster')

In [65]:
bgSrc_weight_raster.save('bgSrcWts')